In [3]:
from deepmol.loaders.loaders import CSVLoader

# load a dataset from a CSV (required fields: dataset_path and smiles_field)
loader = CSVLoader(dataset_path='data8625-2030-2603.csv',
                   #smiles_field='smiles',
                   #id_field='ids',
                   #labels_fields=['n'],
                   features_fields=['Molecule ChEMBL ID', 'Smiles', 'Standard Value'],
                   shard_size=1000,
                   mode='auto')

dataset = loader.create_dataset()

# print shape of the dataset (molecules, X, y)
dataset.get_shape()

((1000,), None, (1000,))

TypeError: __init__() missing 1 required positional argument: 'smiles_field'

In [4]:
def label_by_ic50(molecule):
    # 获取分子的IC50属性，如果没有则返回None
    ic50 = molecule.GetProp('IC50')
    if ic50 is None:
        return None
    # 将IC50属性转换为浮点数
    ic50 = float(ic50)
    # 根据IC50值的范围返回标签
    if ic50 <= 5000:
        return 'Active'
    elif ic50 >= 5001:
        return 'Inactive'
    else:
        return None

In [ ]:
from DeepMol.preprocessing import CompoundFeaturizer

In [5]:
dataset = DeepMol.load_molecules('data8625-2030-2603.csv')

# 使用定义好的函数，对分子数据集进行分类
dataset = DeepMol.label_molecules(dataset, label_by_ic50)

# 查看分子数据集的标签分布
dataset.label_distribution()

NameError: name 'DeepMol' is not defined

现在，你来扮演数据分析师。用deepmol处理一份csv表格，csv里包含了三列：‘CompoundsName’，“Smiles”，‘IC50’。你需要根据IC50值将他们归为两类：Active molecules，inactive molecules。IC50<=5000 nM, 被deepmol归类为Active molecules；IC50>=5001 nM, 被deepmol归类为Active molecules。给出符合以上要求的代码

In [16]:
import pandas as pd
from deepmol import utils
df = pd.read_csv('data8625-2030-2603.csv')
df['IC50'] = pd.to_numeric(df['IC50'], errors='coerce')

def classify_ic50(ic50):
    if ic50 <= 5000:
        return 'Active molecules'
    else:
        return 'Inactive molecules'

df['Classification'] = df['IC50'].apply(classify_ic50)

df.to_csv('palm1_classify',index=False)

In [18]:
import pandas as pd
from deepmol import utils
from deepmol import DeepmolModel

df = pd.read_csv('your_file.csv')
df['IC50'] = pd.to_numeric(df['IC50'], errors='coerce')

def classify_ic50(ic50):
    if ic50 <= 5000:
        return 'Active molecules'
    else:
        return 'Inactive molecules'

df['Classification'] = df['IC50'].apply(classify_ic50)

model = DeepmolModel()
model.fit(df['Smiles'], df['Classification'])
predictions = model.predict(df['Smiles'])

print(predictions)

ImportError: cannot import name 'DeepmolModel' from 'deepmol' (/home/mxc/anaconda3/envs/opencadd/lib/python3.8/site-packages/deepmol/__init__.py)

In [20]:
import pandas as pd
import deepchem as dc

# 读取CSV文件
data_file = "data8625-2030-2603.csv"
dataframe = pd.read_csv(data_file)

# 根据IC50值将分子分为活性分子和非活性分子
dataframe['Class'] = dataframe['IC50'].apply(lambda x: 'Active molecules' if x <= 5000 else 'Inactive molecules')

# 将分子从SMILES字符串转化为DeepMol分子对象
molecules = [dc.Mol.from_smiles(smiles) for smiles in dataframe['Smiles']]

# 对于二分类任务，我们可以使用deepchem库的GraphConvModel
# 首先，我们需要准备数据集
graph_featurizer = dc.feat.ConvMolFeaturizer()
X = graph_featurizer.featurize(molecules)

# 使用'MultiTaskClassifier'将活性/非活性作为目标进行LabelEncoder处理
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(dataframe['Class'])

# 构建数据集：DeepMol对象，活性/非活性（分类结果），阈值为0.5
dataset = dc.data.NumpyDataset(X, y)
splitter = dc.splits.RandomStratifiedSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset)

# 使用图卷积模型进行分类
model = dc.models.GraphConvModel(1, mode='classification', batch_normalize=True)
model.fit(train_dataset)

# 检查分类准确性
metric = dc.metrics.Metric(dc.metrics.accuracy_score)
train_scores = model.evaluate(train_dataset, [metric])
valid_scores = model.evaluate(valid_dataset, [metric])
test_scores = model.evaluate(test_dataset, [metric])

print('Training accuracy: ', train_scores)
print('Validation accuracy: ', valid_scores)
print('Test accuracy: ', test_scores)


2023-04-13 15:44:31.151492: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 15:44:31.374827: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-13 15:44:31.374849: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-13 15:44:31.416002: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-13 15:44:32.238233: W tensorflow/stream_executor/platform/de

AttributeError: module 'deepchem' has no attribute 'Mol'

In [24]:
import numpy as np
from deepchem.utils import *
from deepmol import loaders
from pathlib import Path

# 读取CSV文件
data_file = "data8625-2030-2603.csv"
dataframe = pd.read_csv(data_file)

# 根据IC50值将分子分为活性分子和非活性分子
dataframe['Class'] = dataframe['IC50'].apply(lambda x: 'Active molecules' if x <= 5000 else 'Inactive molecules')

# 使用deepchem将分子从SMILES字符串转化为RDKit的分子对象
molecules = [dc.Mol.from_smiles(smiles) for smiles in dataframe['Smiles']]

# 使用DeepMol的特征提取器生成描述符矩阵
fingerprinter = loader.Fingerprinter(fp_type="topological")
X = fingerprinter.fit_transform(molecules)

# 同样我们需要将活性/非活性作为目标进行LabelEncoder处理
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(dataframe['Class'])

# 使用Scikit-learn库划分训练集和测试集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 使用随机森林分类器进行二分类
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

# 计算模型准确度
train_accuracy = rf.score(X_train, y_train)
test_accuracy = rf.score(X_test, y_test)

print('Training accuracy:', train_accuracy)
print('Test accuracy:', test_accuracy)


AttributeError: module 'deepchem' has no attribute 'Mol'

In [28]:
import pandas as pd
import numpy as np
from deepmol import loaders
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import AllChem

# 读取CSV文件
data_file = "data8625-2030-2603.csv"
dataframe = pd.read_csv(data_file)

# 根据IC50值将分子分为活性分子和非活性分子
dataframe['Class'] = dataframe['IC50'].apply(lambda x: 'Active molecules' if x <= 5000 else 'Inactive molecules')

# 使用RDKit将分子从SMILES字符串转化为RDKit的分子对象
molecules = [Chem.MolFromSmiles(smiles) for smiles in dataframe['Smiles']]

# 使用DeepMol的特征提取器生成描述符矩阵
fingerprinter = loaders.Fingerprinter(fp_type="topological")
X = fingerprinter.fit_transform(molecules)

# 同样我们需要将活性/非活性作为目标进行LabelEncoder处理
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(dataframe['Class'])

# 使用Scikit-learn库划分训练集和测试集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 使用随机森林分类器进行二分类
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

# 计算模型准确度
train_accuracy = rf.score(X_train, y_train)
test_accuracy = rf.score(X_test, y_test)

print('Training accuracy:', train_accuracy)
print('Test accuracy:', test_accuracy)


AttributeError: module 'deepmol.loaders' has no attribute 'Fingerprinter'

In [ ]:
mol = Chem.MolFromSmiles(smiles, sanitize=False)
mol.UpdatePropertyCache(strict=False)
Chem.SanitizeMol(mol, Chem.SANITIZE_ALL ^ Chem.SANITIZE_PROPERTIES, catchErrors=True)